In [1]:
import re
import polars as pl
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import altair as alt
from zenrows import ZenRowsClient


ModuleNotFoundError: No module named 'zenrows'

In [2]:
# pip install zenrows
client = ZenRowsClient("76478ec118fc9e726b3c8dafcd9bfc832cf20d42")
url = "https://www.flysfo.com/passengers/parking"

response = client.get(url)

print(response.text)


ModuleNotFoundError: No module named 'zenrows'

In [12]:
response = requests.get(
    "https://www.flysfo.com/passengers/parking", proxies=proxy_addresses
)
soup = BeautifulSoup(response.content, "html.parser")
print(soup.prettify())

ProxyError: HTTPSConnectionPool(host='www.flysfo.com', port=443): Max retries exceeded with url: /passengers/parking (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 503 Too many open connections')))

In [3]:

gc = soup.find(class_="garageCard")
print(gc.attrs)

AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:

garage_card_title = gc.find("div", class_="garageCard__title")
print(garage_card_title.text)

 Domestic Garage


In [ ]:
gc_all = soup.find_all(class_="garageCard")


In [ ]:
current_time_pst = datetime.now()
current_date = current_time_pst.strftime("%Y-%m-%d")
current_time = current_time_pst.strftime("%H:%M:%S")

dat = pl.DataFrame(
    {
        "date": [current_date for _ in gc_all],
        "time": [current_time for _ in gc_all],
        "garage": [_.find("div", class_="garageCard__title").text.strip() for _ in gc_all],
        "spots": [
            int(
                re.search(
                    r"\d+", _.find("p", class_="garageCard__infos__spots").text.strip()
                ).group()
            )
            for _ in gc_all
        ],
        "occupancy": [
            int(
                re.search(
                    r"\d+", _.find("text", class_="percentage").text.strip()
                ).group()
            )
            for _ in gc_all
        ],
    }
)
print(dat)

shape: (5, 5)
┌────────────┬──────────┬────────────────────────┬───────┬───────────┐
│ date       ┆ time     ┆ garage                 ┆ spots ┆ occupancy │
│ ---        ┆ ---      ┆ ---                    ┆ ---   ┆ ---       │
│ str        ┆ str      ┆ str                    ┆ i64   ┆ i64       │
╞════════════╪══════════╪════════════════════════╪═══════╪═══════════╡
│ 2024-12-12 ┆ 04:15:31 ┆ Domestic Garage        ┆ 1180  ┆ 76        │
│ 2024-12-12 ┆ 04:15:31 ┆ International Garage A ┆ 578   ┆ 65        │
│ 2024-12-12 ┆ 04:15:31 ┆ International Garage G ┆ 493   ┆ 65        │
│ 2024-12-12 ┆ 04:15:31 ┆ ParkFAST               ┆ 106   ┆ 55        │
│ 2024-12-12 ┆ 04:15:31 ┆ Long Term Garage       ┆ 2652  ┆ 60        │
└────────────┴──────────┴────────────────────────┴───────┴───────────┘


In [5]:
df = pl.read_csv("/workspaces/bayarea_airport_parking/data/data.csv")
df

date,time,garage,spots,occupancy
str,str,str,i64,i64
"""2024-12-11""","""21:29:01""",""" Domestic Garage""",1149,77
"""2024-12-11""","""21:29:01""",""" International Garage A""",669,59
"""2024-12-11""","""21:29:01""",""" International Garage G""",570,59
"""2024-12-11""","""21:29:01""",""" ParkFAST""",107,55
"""2024-12-11""","""21:29:01""",""" Long Term Garage""",2678,60
…,…,…,…,…
"""2024-12-15""","""20:08:40""",""" Domestic Garage""",2233,55
"""2024-12-15""","""20:08:40""",""" International Garage A""",550,67
"""2024-12-15""","""20:08:40""",""" International Garage G""",658,53


In [6]:
df = df.with_columns(
    (
        pl.concat_str([pl.col("date"), pl.col("time")], separator=" ")
        .str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S")
        .alias("datetime")
    )
)
df

date,time,garage,spots,occupancy,datetime
str,str,str,i64,i64,datetime[μs]
"""2024-12-11""","""21:29:01""",""" Domestic Garage""",1149,77,2024-12-11 21:29:01
"""2024-12-11""","""21:29:01""",""" International Garage A""",669,59,2024-12-11 21:29:01
"""2024-12-11""","""21:29:01""",""" International Garage G""",570,59,2024-12-11 21:29:01
"""2024-12-11""","""21:29:01""",""" ParkFAST""",107,55,2024-12-11 21:29:01
"""2024-12-11""","""21:29:01""",""" Long Term Garage""",2678,60,2024-12-11 21:29:01
…,…,…,…,…,…
"""2024-12-15""","""20:08:40""",""" Domestic Garage""",2233,55,2024-12-15 20:08:40
"""2024-12-15""","""20:08:40""",""" International Garage A""",550,67,2024-12-15 20:08:40
"""2024-12-15""","""20:08:40""",""" International Garage G""",658,53,2024-12-15 20:08:40


In [7]:

source = df

nearest = alt.selection_point(
    nearest=True, on="pointerover", fields=["datetime"], empty=False
)

line = (
    alt.Chart(data=source)
    .encode(x="datetime", y="spots", color="garage")
    .mark_line()
    .interactive()
)

selectors = (
    alt.Chart(source)
    .mark_point()
    .encode(
        x="datetime",
        opacity=alt.value(0),
    )
    .add_params(nearest)
)
when_near = alt.when(nearest)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=when_near.then(alt.value(1)).otherwise(alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align="left", dx=5, dy=-5).encode(
    text=when_near.then("spots:Q").otherwise(alt.value(" "))
)

# Draw a rule at the location of the selection
rules = (
    alt.Chart(source)
    .mark_rule(color="gray")
    .encode(
        x="datetime",
    )
    .transform_filter(nearest)
)
# Put the five layers into a chart and bind the data
alt.layer(line, selectors, points, rules, text).properties(width=600, height=300)

alt.LayerChart(...)

In [8]:
source = df

nearest = alt.selection_point(
    nearest=True, on="pointerover", fields=["datetime"], empty=False
)

line = (
    alt.Chart(data=source)
    .encode(x="datetime", y="occupancy", color="garage")
    .mark_line()
    .interactive()
)

selectors = (
    alt.Chart(source)
    .mark_point()
    .encode(
        x="datetime",
        opacity=alt.value(0),
    )
    .add_params(nearest)
)
when_near = alt.when(nearest)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=when_near.then(alt.value(1)).otherwise(alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align="left", dx=5, dy=-5).encode(
    text=when_near.then("occupancy:Q").otherwise(alt.value(" "))
)

# Draw a rule at the location of the selection
rules = (
    alt.Chart(source)
    .mark_rule(color="gray")
    .encode(
        x="datetime",
    )
    .transform_filter(nearest)
)
# Put the five layers into a chart and bind the data
alt.layer(line, selectors, points, rules, text).properties(width=600, height=300)


alt.LayerChart(...)